# Installation
Python 3.9 is required for this assignment.
To install the required packages, create a new `pip` virtual environment and install the required libraries by running
```
pip install -r requirements.txt
```

If you encounter issues with the python version or the installation of requirements.txt, please reach out to `konstantinos.kitsios@uzh.ch`.

# Setup
The first part of the notebook sets up the environment by implementing classes/functions we talked about during the lecture.

In [5]:
from urllib.parse import urlparse
import random
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
import pickle
import hashlib
from typing import Tuple, List, Callable, Set, Any

In [6]:
from assignment_utils import Fuzzer, Runner, FunctionRunner, FunctionCoverageRunner, Coverage, population_coverage, Location

Remember the `urlparse()` function from the lecture? We will use this again. You might also remember that it never crashes, even with invalid urls, so we will use the corresponding `urlparse_that_crashes()`, like we did in the lecture.

In [7]:
url = "https://uzh.ch"
urlparse(url)

ParseResult(scheme='https', netloc='uzh.ch', path='', params='', query='', fragment='')

In [8]:
def urlparse_that_crashes(url: str) -> bool:
    supported_schemes = ["http", "https"]
    result = urlparse(url)
    if result.scheme not in supported_schemes:
        raise ValueError("Scheme must be one of " + repr(supported_schemes))
    if result.netloc == '':
        raise ValueError("Host must be non-empty")

    return True

Now, we will copy the code of the second live coding session we did in during the lecture. The code below implements the mutation part of a mutation-based blackbox fuzzer.

In [9]:
def mutate(s, num_of_mutations = 1):
    mutators = [delete_random, insert_random, flip_random]

    for i in range(num_of_mutations):
        mutator_ind = random.randint(0, 2)
        selected_mutator = mutators[mutator_ind]
        s = selected_mutator(s)

    return s

In [10]:
def delete_random(s):
    pos_to_delete = random.randint(0, len(s) - 1)
    return s[:pos_to_delete-1] + s[pos_to_delete:]

In [11]:
def insert_random(s):
    pos_to_insert = random.randint(0, len(s)-1)
    char_to_insert = chr(random.randint(32, 64))
    return s[:pos_to_insert-1] + char_to_insert + s[pos_to_insert-1:]

In [12]:
def flip_random(s):
    pos_to_flip  = random.randint(0, len(s)-1)
    byte_to_flip = ord(s[pos_to_flip]) # char --> byte
    bit_to_flip  = 1 << random.randint(0, 6) # 0010000 or 100000
    flipped_char = chr(byte_to_flip ^ bit_to_flip)

    return s[:pos_to_flip] + flipped_char + s[pos_to_flip+1:]

To put the function `mutate()` in use, we will implement the mutation-based blackbox fuzzer in the `MutationBlackboxFuzzer` class as following. The base class `Fuzzer` is just a placeholder that you can find in `utils.py`.

In [13]:
class MutationBlackboxFuzzer(Fuzzer):
    """Base class for mutational fuzzing"""

    def __init__(self, seed: List[str],
                 min_mutations: int = 2,
                 max_mutations: int = 10) -> None:
        """Constructor.
        `seed` - a list of (input) strings to mutate.
        `min_mutations` - the minimum number of mutations to apply.
        `max_mutations` - the maximum number of mutations to apply.
        """
        self.seed = seed
        self.min_mutations = min_mutations
        self.max_mutations = max_mutations
        self.reset()

    def reset(self) -> None:
        """Set population to initial seed.
        To be overloaded in subclasses."""
        self.population = self.seed
        self.seed_index = 0


    def mutate(self, inp: str) -> str:
        return mutate(inp)

    def create_candidate(self) -> str:
        """Create a new candidate by mutating a population member"""
        candidate = random.choice(self.population)
        trials = random.randint(self.min_mutations, self.max_mutations)
        for i in range(trials):
            candidate = self.mutate(candidate)
        return candidate

    def fuzz(self) -> str:
        # If we did not return all the seeds, return the next seed. Only after we run 
        # out of seeds, we return a mutated version of the seed
        if self.seed_index < len(self.seed):
            # Still seeding
            self.inp = self.seed[self.seed_index]
            self.seed_index += 1
        else:
            # Mutating
            self.inp = self.create_candidate()
        return self.inp

In [14]:
seed_input = "http://www.google.com/search?q=fuzzing"
mutation_fuzzer = MutationBlackboxFuzzer(seed=[seed_input])

Remember the `Runner` class we used in the lecture to measure the coverage achieved by a specific input. Instead of calling
```python
result = urlparse_that_crashes("https://uzh.ch")
```

you can instead do
```python
http_runner = FunctionCoverageRunner(urlparse_that_crashes) # define the function here
result, isCrash = http_runner.run("https://uzh.ch") # define the input here
```
which also gives you access to the coverage achieved:
```
list(http_runner.coverage())
```

In [15]:
http_runner = FunctionCoverageRunner(urlparse_that_crashes) # Only initialize it once per function!
http_runner.run("https://foo.bar/")

len(list(http_runner.coverage()))

50

Putting it all together, we can now run our fuzzer with a single seed for 1000 trials and measure the number of covered lines  like this

In [16]:
seed_input = "https://uzh.ch"
mutation_fuzzer = MutationBlackboxFuzzer(seed=[seed_input])

mutation_fuzzer.runs(http_runner, trials=1000)
print(mutation_fuzzer.population)

print("%d lines covered" % len(http_runner.coverage()))

['https://uzh.ch']
42 lines covered


# Task 01 - Investigating the Effect of Starting Seeds
In the above run, our fuzzer covers 40 lines using only one starting seed ("https://uzh.ch"). Your first task is to investigate the effect of the starting seed in fuzzer performance. We remind that fuzzer performance is usually measure by the achieved coverage of the fuzzer (higher coverage => higher probability of exposing bugs => better performance).

Compare the coverage of the `MutationBlackboxFuzzer` with one seed vs ten seeds. The choice of the ten seeds is up to you, you can either handpick ten random urls, or write a small script that randomly navigates the web and returns ten urls for more diversity.

Comparing the performance of two fuzzers (or the same fuzzer with different configurations in our case) is a nuanced topic due to the inherent randomness of fuzzing. For example, if we run the `MutationBlackboxFuzzer` with one seed and cover 40 lines, and then run it with ten seeds and cover 42 lines, can we claim that the more starting seeds the better? Probably not.
To make claims based on empirical data, we have to run statistical tests. In fuzzing, typically each fuzzer configuration is ran for 30 runs of 10,000 trials (i.e., you run the above cell 30 times), yielding an array of 30 coverage values for each fuzzer. The two arrays are then compared using the Mann-Whitney U Test with the null hypothesis that the values of the first array (i.e., coverage of the first fuzzer) are greater than those of the second array. If the p-value of the test is <0.05, we reject the null hypothesis, claiming that the first fuzzer achieves less coverage than the second one (with confidence level 1-0.05 = 95%).
You can run the Mann-Whitney U Test in python with the following code:
```python
from scipy.stats import mannwhitneyu
u_stat, p_value = mannwhitneyu(c1, c2, alternative='less')
if p_value < 0.05:
    print("Values of c1 are smaller than those in c2 with confidence level 95%")
```

In [17]:
# Your code here

from scipy.stats import mannwhitneyu

# Definition of the 10 starting seeds
seeds = [
    "https://uzh.ch",
    "https://ethz.ch",
    "https://example.com",
    "https://foo.bar",
    "https://bar.baz",
    "https://test.site",
    "https://abc.def",
    "https://random.url",
    "https://another.test",
    "https://yet.another"
]

# We run the fuzzer for each seed and collect coverage results
coverage_results = []

for seed_input in seeds: # For every seed, we do a run with 10000 trials
    mutation_fuzzer = MutationBlackboxFuzzer(seed=[seed_input]) # We create a new fuzzer
    mutation_fuzzer.runs(http_runner, trials=10000)  # We do 10000 trials
    covered_lines = len(list(http_runner.coverage()))
    coverage_results.append(covered_lines)

# Print the coverage results for the 10 URLs
print("Coverage results for each seed:")
for seed, coverage in zip(seeds, coverage_results):
    print(f"{seed}: {coverage} lines covered")


# Split into two groups: first 5 seeds and last 5 seeds
group1 = coverage_results[:5]
group2 = coverage_results[5:]

# Perform the Mann-Whitney U test
u_stat, p_value = mannwhitneyu(group1, group2, alternative='less')

# Print results from Mann-Whitney U test
print("\nMann-Whitney U Test Result:")
print(f"U-statistic: {u_stat}")
print(f"P-value: {p_value}")

if p_value < 0.05:
    print("Values of group 1 are smaller than those in group 2 with 95% confidence.")
else:
    print("No significant difference detected between the two groups.")


Coverage results for each seed:
https://uzh.ch: 39 lines covered
https://ethz.ch: 46 lines covered
https://example.com: 50 lines covered
https://foo.bar: 43 lines covered
https://bar.baz: 50 lines covered
https://test.site: 39 lines covered
https://abc.def: 40 lines covered
https://random.url: 43 lines covered
https://another.test: 42 lines covered
https://yet.another: 43 lines covered

Mann-Whitney U Test Result:
U-statistic: 19.5
P-value: 0.944753987972166
No significant difference detected between the two groups.


# Task 02: Investigating the Effect of More Sophisticated Mutations
The mutation function we implemented above (and in the lecture) is:
```python
def mutate(s, num_of_mutations = 1):
    mutators = [delete_random, insert_random, flip_random]

    for i in range(num_of_mutations):
        mutator_ind = random.randint(0, 2)
        selected_mutator = mutators[mutator_ind]
        s = selected_mutator(s)

    return s
```

Your task is to implement a more sophisticated mutation strategy by:
1. Extending the `mutate(s, num_of_mutations = 1)` function above to support three more mutators
    - Addition of small integers to a random byte
    - Replacing a random byte with a completely random byte value
    - Block duplucation (insert a random block of characters of `s` in a random position of `s`)
2. Extending the `mutate(self, s)` method of `MutationBlackboxFuzzer` so that with probability `p` it calls the `mutate(s, num_of_mutations = 1)` function above and with probability `1-p` mutates the input using **splicing**; Splicing (a.k.a, crossover) refers to selecting another seed from the existing population, and merging a random block of the new seed with the current seed at a random position.

Compare the performance of your new fuzzer with the previous fuzzer over 30 runs of 5000 trials each. Use a single starting seed `s=["https://www.swissinfo.ch"]`. What can you claim about the effect of more complex mutations?

Note: For more information about the exact mutations used in the most popular fuzzer, AFL, you can have a look at this blog by the AFL creator: https://lcamtuf.blogspot.com/2014/08/binary-fuzzing-strategies-what-works.html

In [18]:
# Your code here

def add_small_int(s):
    if len(s) == 0:
        return s
    pos = random.randint(0, len(s) - 1)
    orig_char = s[pos]
    new_char = chr((ord(orig_char) + random.randint(1, 10)) % 256)
    return s[:pos] + new_char + s[pos + 1:]
    
def replace_random(s):
    if len(s) == 0:
        return s
    pos = random.randint(0, len(s) - 1)
    new_char = chr(random.randint(0, 255))
    return s[:pos] + new_char + s[pos + 1:]

def block_duplicate(s):
    if len(s) < 2:
        return s
    start = random.randint(0, len(s) - 2)
    end = random.randint(start + 1, len(s))
    block = s[start:end]
    insert_pos = random.randint(0, len(s))
    return s[:insert_pos] + block + s[insert_pos:]

def mutate(s, num_of_mutations = 1):
    mutators = [delete_random, insert_random, flip_random, add_small_int, replace_random, block_duplicate]

    for i in range(num_of_mutations):
        mutator_ind = random.randint(0, 5)
        selected_mutator = mutators[mutator_ind]
        s = selected_mutator(s)

    return s

class MutationBlackboxFuzzer2(MutationBlackboxFuzzer):
    def __init__(self, seed, min_mutations=2, max_mutations=10, p=0.5):
        super().__init__(seed, min_mutations, max_mutations)
        self.p = p  # Probability of using standard mutation

    def mutate(self, s: str) -> str:
        if random.random() < self.p:
            # Standard mutation
            return mutate(s, num_of_mutations=1)
        else:
            # Splicing mutation
            if not self.population:
                return s  # fallback

            other = random.choice(self.population)
            if len(other) < 2:
                return s  # Not enough to splice from

            start = random.randint(0, len(other) - 2)
            end = random.randint(start + 1, len(other))
            block = other[start:end]

            insert_pos = random.randint(0, len(s))
            return s[:insert_pos] + block + s[insert_pos:]

# Test
import random
from scipy.stats import mannwhitneyu

# setup
seed = ["https://www.swissinfo.ch"]
runs = 30
trials = 5000

coverage_1 = []
coverage_2 = []

# Run both fuzzers
for i in range(runs):
    # Old fuzzer
    fuzzer1 = MutationBlackboxFuzzer(seed=[seed_input])
    fuzzer1.runs(http_runner, trials=trials)
    coverage_1.append(len(http_runner.coverage()))

    # New fuzzer
    fuzzer2 = MutationBlackboxFuzzer2(seed=[seed_input], p=0.5)
    fuzzer2.runs(http_runner, trials=trials)
    coverage_2.append(len(http_runner.coverage()))

u_stat, p_value = mannwhitneyu(coverage_1, coverage_2, alternative='less')

print("Old Fuzzer Coverage (avg):", sum(coverage_1) / len(coverage_1))
print("New Fuzzer Coverage (avg):", sum(coverage_2) / len(coverage_2))
print("\nMann-Whitney U Test:")
print("U statistic:", u_stat)
print("p-value:", p_value)

if p_value < 0.05:
    print("Fuzzer2 performs significantly better.")
else:
    print("No significant difference detected.")



Old Fuzzer Coverage (avg): 43.5
New Fuzzer Coverage (avg): 48.2

Mann-Whitney U Test:
U statistic: 217.5
p-value: 0.00020402968952283
Fuzzer2 performs significantly better.


# Task 03: Compare With Mutation-Based Greybox Fuzzer
To improve on the blackbox fuzzer in the lecture, we implemented the greybox fuzzer below, that dynamically adds a mutated input to the seeds queue if that input covers a new path. 

Your task is to compare the performance of the (already implemented) greybox fuzzer with that of the blackbox fuzzer over 30 runs of 5000 trials each.
Use the same single starting seed as the previous experiment.

What can you claim about the performance of they greybox fuzzer?

In [19]:
class MutationGreyboxFuzzer(MutationBlackboxFuzzer):
    """Fuzz with mutated inputs based on coverage"""

    def reset(self) -> None:
        super().reset()
        self.coverages_seen: Set[frozenset] = set()
        # Now empty; we fill this with seed in the first fuzz runs
        self.population = []

    def run(self, runner: FunctionCoverageRunner) -> Any:
        """Run function(inp) while tracking coverage.
           If we reach new coverage,
           add inp to population and its coverage to population_coverage
        """
        result, outcome = super().run(runner)
        new_coverage = frozenset(runner.coverage())
        if outcome == Runner.PASS and new_coverage not in self.coverages_seen:
            # We have new coverage
            self.population.append(self.inp)
            self.coverages_seen.add(new_coverage)

        return result

In [20]:
# Your code here

# Test
import random
from scipy.stats import mannwhitneyu

# setup
seed = ["https://www.swissinfo.ch"]
runs = 30
trials = 5000

coverage_blackbox = []
coverage_greybox = []

# Run both fuzzers
for i in range(runs):
    # Blackbox fuzzer
    fuzzer_blackbox = MutationBlackboxFuzzer(seed=[seed_input])
    fuzzer_blackbox.runs(http_runner, trials=trials)
    coverage_blackbox.append(len(http_runner.coverage()))

    # Greybox fuzzer
    fuzzer_greybox = MutationGreyboxFuzzer(seed=[seed_input])
    fuzzer_greybox.runs(http_runner, trials=trials)
    coverage_greybox.append(len(http_runner.coverage()))

u_stat, p_value = mannwhitneyu(coverage_blackbox, coverage_greybox, alternative='less')

print("Blackbox Fuzzer Coverage (avg):", sum(coverage_blackbox) / len(coverage_blackbox))
print("Greybox Fuzzer Coverage (avg):", sum(coverage_greybox) / len(coverage_greybox))
print("\nMann-Whitney U Test:")
print("U statistic:", u_stat)
print("p-value:", p_value)

if p_value < 0.05:
    print("Greybox performs significantly better.")
else:
    print("No significant difference detected.")



Blackbox Fuzzer Coverage (avg): 44.06666666666667
Greybox Fuzzer Coverage (avg): 54.766666666666666

Mann-Whitney U Test:
U statistic: 102.5
p-value: 1.3020656552619943e-07
Greybox performs significantly better.


# Task 04: Boosted Greybox Fuzzer
In page 30 of the lecture materials, we discussed about boosting the performance of greybox fuzzers by prioritizing seeds that exercise more rare paths. Your task is to implement this into a class named `BoostedMutationGreyboxFuzzer` and compare its performance against the `MutationGreyboxFuzzer`.

Specifically, you must calculate the coverage achieved for each input you feed into the program-under-test, and keep track of how frequently a specific coverage is triggered (hint: you may find the function `getPathID()` below useful for this). 

Then, when selecting the next seed to mutate, you must select with probability reversly proportional to the frequency of the coverage of each seed. As a result, seeds that exercise rarer paths will be selected more frequently.

Compare the performance of your new `BoostedMutationGreyboxFuzzer` with the base `MutationGreyboxFuzzer` in two target programs:
1. On the `urlparse_that_crashes()` program using 30 runs of 5,000 trials each and the same single starting seed as above.
2. On the `crashme()` program found below, 30 runs of using 10,000 trials and a single starting seed `s=["good"]`.

In [21]:
def getPathID(coverage: Any) -> str:
    """Returns a unique hash for the covered statements"""
    pickled = pickle.dumps(sorted(coverage))
    return hashlib.md5(pickled).hexdigest()

In [22]:
def crashme(s: str) -> None:
    if len(s) > 0 and s[0] == 'b':
        if len(s) > 1 and s[1] == 'a':
            if len(s) > 2 and s[2] == 'd':
                if len(s) > 3 and s[3] == '!':
                    raise Exception()

In [23]:
class BoostedMutationGreyboxFuzzer(MutationBlackboxFuzzer):
    """Fuzz with mutated inputs based on coverage"""

    def reset(self) -> None:
        super().reset()
        self.coverages_seen: Set[frozenset] = set()
        # Now empty; we fill this with seed in the first fuzz runs
        self.population = []
        super().reset()
        self.path_frequencies = {}  # Dictionary to track path frequencies

    def run(self, runner: FunctionCoverageRunner) -> Any:
        result, outcome = super().run(runner)
        new_coverage = frozenset(runner.coverage())
        path_id = getPathID(new_coverage)

        # Update path frequency
        if path_id not in self.path_frequencies:
            self.path_frequencies[path_id] = 0
        self.path_frequencies[path_id] += 1

        return result

    def create_candidate(self) -> str:
        if not self.population:
            return super().create_candidate()

        # Calculate selection probabilities
        weights = []
        for seed in self.population:
            runner = FunctionCoverageRunner(lambda x: None)  # Dummy runner
            runner.run(seed)
            path_id = getPathID(frozenset(runner.coverage()))
            frequency = self.path_frequencies.get(path_id, 1)
            weights.append(1 / frequency)

        # Normalize weights
        total_weight = sum(weights)
        probabilities = [w / total_weight for w in weights]

        # Select a seed based on probabilities
        selected_seed = random.choices(self.population, weights=probabilities, k=1)[0]

        # Mutate the selected seed
        return self.mutate(selected_seed)

In [24]:
import random
from scipy.stats import mannwhitneyu

# Function to compare two fuzzers
def compare_fuzzers(fuzzer1_class, fuzzer2_class, runner, seed, trials, runs):
    coverage_fuzzer1 = []
    coverage_fuzzer2 = []

    for _ in range(runs):
        # Run fuzzer 1
        fuzzer1 = fuzzer1_class(seed=[seed])
        fuzzer1.runs(runner, trials=trials)
        coverage_fuzzer1.append(len(runner.coverage()))

        # Run fuzzer 2
        fuzzer2 = fuzzer2_class(seed=[seed])
        fuzzer2.runs(runner, trials=trials)
        coverage_fuzzer2.append(len(runner.coverage()))

    # Perform Mann-Whitney U Test
    u_stat, p_value = mannwhitneyu(coverage_fuzzer1, coverage_fuzzer2, alternative='less')

    # Print results
    print(f"Fuzzer 1 (avg coverage): {sum(coverage_fuzzer1) / len(coverage_fuzzer1)}")
    print(f"Fuzzer 2 (avg coverage): {sum(coverage_fuzzer2) / len(coverage_fuzzer2)}")
    print(f"Mann-Whitney U Test: U-statistic={u_stat}, p-value={p_value}")
    if p_value < 0.05:
        print("Fuzzer 2 performs significantly better.")
    else:
        print("No significant difference detected.")

# Target 1: urlparse_that_crashes()
print("Comparing on urlparse_that_crashes()")
url_runner = FunctionCoverageRunner(urlparse_that_crashes)
compare_fuzzers(
    MutationGreyboxFuzzer,
    BoostedMutationGreyboxFuzzer,
    url_runner,
    seed="https://uzh.ch",
    trials=5000,
    runs=30
)

# Target 2: crashme()
print("\nComparing on crashme()")
crashme_runner = FunctionCoverageRunner(crashme)
compare_fuzzers(
    MutationGreyboxFuzzer,
    BoostedMutationGreyboxFuzzer,
    crashme_runner,
    seed="good",
    trials=10000,
    runs=30
)

Comparing on urlparse_that_crashes()
Fuzzer 1 (avg coverage): 51.93333333333333
Fuzzer 2 (avg coverage): 47.333333333333336
Mann-Whitney U Test: U-statistic=586.5, p-value=0.9789604516774981
No significant difference detected.

Comparing on crashme()
Fuzzer 1 (avg coverage): 2.6333333333333333
Fuzzer 2 (avg coverage): 2.0
Mann-Whitney U Test: U-statistic=615.0, p-value=0.9998578683768669
No significant difference detected.


# Remark
For Task 04, we used GitHub Copilot to assist in developing a functioning implementation of the BoostedMutationGreyboxFuzzer class. The AI tool was used to support and accelerate the coding process, but the final implementation was reviewed and adapted to fit the specific requirements of the task.